# Personalized UI Recommendation System - Colab Version

## 1. 패키지 설치 및 프로젝트 클론

In [ ]:
# 필수 패키지 설치
!pip install lightgbm pandas numpy scikit-learn joblib pyyaml tqdm
    
# 코랩 파일 다운로드 활성화
from google.colab import files
    
import sys
import os

# /content로 이동 
%cd /content
print("현재 디렉토리:", os.getcwd())

# 기존 디렉토리 삭제
!rm -rf Personalized-UI-Recommendation-System

# 다시 클론
!git clone https://github.com/jud1thDev/Personalized-UI-Recommendation-System.git

# 프로젝트 디렉토리로 이동
%cd Personalized-UI-Recommendation-System

# 경로 설정
import sys
COLAB_PROJECT_PATH = '/content/Personalized-UI-Recommendation-System'
sys.path.append(COLAB_PROJECT_PATH)

print("프로젝트 경로:", os.getcwd())
print("Python 경로 설정 완료")
    
# 코랩에서 필요한 디렉토리 생성
os.makedirs("ui_rec/data/raw", exist_ok=True)
os.makedirs("ui_rec/data/processed", exist_ok=True)
os.makedirs("ui_rec/data/models", exist_ok=True)
os.makedirs("ui_rec/data/outputs", exist_ok=True)
    
print("코랩 환경 설정 완료")

In [ ]:
# 필수 패키지 설치
%pip install -r requirements.txt
    
import sys
import os
    
# 로컬 환경: 현재 프로젝트 사용
LOCAL_PROJECT_PATH = os.getcwd()
sys.path.append(LOCAL_PROJECT_PATH)
    
print(f"로컬 환경에서 실행 중. 프로젝트 경로: {LOCAL_PROJECT_PATH}")
    
# 로컬에서 필요한 디렉토리 확인/생성
os.makedirs("ui_rec/data/raw", exist_ok=True)
os.makedirs("ui_rec/data/processed", exist_ok=True)
os.makedirs("ui_rec/data/models", exist_ok=True)
os.makedirs("ui_rec/data/outputs", exist_ok=True)
    
print("로컬 환경 설정 완료")

## 2. 모듈 import 및 설정

In [ ]:
import pandas as pd
import numpy as np
import joblib
import warnings
from datetime import datetime, timedelta
import random
import yaml
import os

warnings.filterwarnings('ignore')

from ui_rec.src.data.generate_mock import generate_users, generate_functions, simulate_events
from ui_rec.src.features.build_features import build_features_colab
from ui_rec.src.models.common import train_binary, train_multiclass, train_regression, split_xy_colab, predict_all_models_colab, predict_exposure_colab, predict_ui_type_colab, predict_service_cluster_colab, predict_rank_colab
from ui_rec.src.models.ui_grouping import create_ui_component_groups

from ui_rec.src.utils.io import write_csv_with_timestamp, ensure_dir, latest_file, save_model
from ui_rec.src.utils.time import access_time_cluster
from ui_rec.src.utils.features import entropy
from ui_rec.src.utils.icons import get_icon_suggestion
from ui_rec.src.utils.component_mapping import get_component_type_by_user_type

print("모듈 import 완료")

## 3. 설정 파일 생성

In [ ]:
def create_config_files():
    # data.yaml 생성
    data_config = {
        "paths": {
            "raw_dir": "ui_rec/data/raw",
            "processed_dir": "ui_rec/data/processed", 
            "models_dir": "ui_rec/data/models",
            "logs_dir": "ui_rec/data/logs",
            "outputs_dir": "ui_rec/data/outputs"
        },
        "files": {
            "events_pattern": "events_*.csv",
            "features_pattern": "features_*.csv",
            "model_pattern": "*.joblib",
            "log_pattern": "*.log",
            "output_pattern": "ui_home_outputs.json"
        }
    }
    
    # model.yaml 생성
    model_config = {
        "lgbm": {
            "exposure": {
                "objective": "binary",
                "metric": "auc",
                "boosting_type": "gbdt",
                "num_leaves": 31,
                "learning_rate": 0.05,
                "feature_fraction": 0.9,
                "bagging_fraction": 0.8,
                "bagging_freq": 5,
                "verbose": -1,
                "num_boost_round": 1000,
                "early_stopping_rounds": 50
            },
            "ui_type": {
                "objective": "multiclass",
                "metric": "multi_logloss",
                "num_class": 5,
                "boosting_type": "gbdt",
                "num_leaves": 31,
                "learning_rate": 0.05,
                "feature_fraction": 0.9,
                "bagging_fraction": 0.8,
                "bagging_freq": 5,
                "verbose": -1,
                "num_boost_round": 1000,
                "early_stopping_rounds": 50
            },
            "service_cluster": {
                "objective": "multiclass",
                "metric": "multi_logloss",
                "num_class": 7,
                "boosting_type": "gbdt",
                "num_leaves": 31,
                "learning_rate": 0.05,
                "feature_fraction": 0.9,
                "bagging_fraction": 0.8,
                "bagging_freq": 5,
                "verbose": -1,
                "num_boost_round": 1000,
                "early_stopping_rounds": 50
            },
            "rank": {
                "objective": "regression",
                "metric": "rmse",
                "boosting_type": "gbdt",
                "num_leaves": 31,
                "learning_rate": 0.05,
                "feature_fraction": 0.9,
                "bagging_fraction": 0.8,
                "bagging_freq": 5,
                "verbose": -1,
                "num_boost_round": 1000,
                "early_stopping_rounds": 50
            }
        },
        "ui": {
            "allowed_types": ["card", "list_item", "banner", "icon", "grid_item"],
            "default_type": "card"
        },
        "service_clusters": {
            "mapping": {
                "account": 0, "finance": 1, "lifestyle": 2, 
                "health": 3, "shopping": 4, "travel": 5, "security": 6
            },
            "labels": {
                "0": "account", "1": "finance", "2": "lifestyle",
                "3": "health", "4": "shopping", "5": "travel", "6": "security"
            }
        }
    }
    
    # 디렉토리 생성
    for path in data_config["paths"].values():
        os.makedirs(path, exist_ok=True)
    
    return data_config, model_config

# 설정 파일 생성 및 전역 변수로 저장
data_config, model_config = create_config_files()
print("설정 파일 생성 완료")

## 4. mock 데이터 생성

In [ ]:
print("모의 데이터 생성 중...")
users = generate_users(n_users=20)
funcs = generate_functions()
events = simulate_events(users, funcs, days=7)

# 데이터 저장
raw_file = write_csv_with_timestamp(events, "user_events", "ui_rec/data/raw")
print(f"모의 데이터 생성 완료: {len(users)}명 사용자, {len(events)}개 이벤트")
print(f"저장 위치: {raw_file}")

## 5. 피처 생성

In [ ]:
print("피처 생성 중...")
processed_file = build_features_colab(raw_file)
print(f"피처 생성 완료")
print(f"저장 위치: {processed_file}")

## 6. 모델 학습

In [ ]:
print("모델 학습 중...")

# 데이터 로드 및 전처리
df = pd.read_csv(processed_file)

# 학습/검증 데이터 분할 (모듈화된 함수 사용)
X, y_exposure = split_xy_colab(df, "exposure_label")
_, y_ui_type = split_xy_colab(df, "ui_type_label")
_, y_service_cluster = split_xy_colab(df, "service_cluster_label")
_, y_rank = split_xy_colab(df, "rank_label")

# Exposure 모델 학습
print("- Exposure 모델 학습 중...")
exposure_model = train_binary(df, "exposure_label", "exposure")
print(f"Exposure 모델 저장 완료")

# UI Type 모델 학습
print("- UI Type 모델 학습 중...")
ui_type_model = train_multiclass(df, "ui_type_label", "ui_type", "ui_type")
print(f"UI Type 모델 저장 완료")

# Service Cluster 모델 학습
print("- Service Cluster 모델 학습 중...")
service_cluster_model = train_multiclass(df, "service_cluster_label", "service_cluster", "service_cluster")
print(f"Service Cluster 모델 저장 완료")

# Rank 모델 학습
print("- Rank 모델 학습 중...")
rank_model = train_regression(df, "rank_label", "rank")
print(f"Rank 모델 저장 완료")

print("모델 학습 완료")

## 7. 추론 및 결과 생성

In [ ]:
print("추론 및 결과 생성 중...")

# 학습 때와 동일한 피처 선택(ID들 제외)
drop_cols = ["ui_type_label", "service_cluster_label", "rank_label"]
target_cols = ["exposure_label"]
feature_cols = [c for c in df.columns 
                if c not in (target_cols + drop_cols + ["user_id", "function_id"])]

print(f"사용할 피처 개수: {len(feature_cols)}")
print(f"피처 컬럼: {feature_cols}")

X_pred = df[feature_cols].copy()

# 모듈화된 함수로 예측
predictions = predict_all_models_colab("ui_rec/data/models", X_pred)

# 예측 결과를 1차원으로 변환
def to_1d_labels(x):
    """(n,), (n,1), (n,k) 모두 (n,) 라벨 벡터로 변환"""
    if isinstance(x, pd.DataFrame):
        arr = x.values
    else:
        arr = np.asarray(x)
    if arr.ndim == 2:
        if arr.shape[1] == 1:
            arr = arr[:, 0]
        else:
            arr = arr.argmax(axis=1)
    return arr.reshape(-1)

exposure_pred         = to_1d_labels(predictions["exposure"])
ui_type_pred          = to_1d_labels(predictions["ui_type"])
service_cluster_pred  = to_1d_labels(predictions["service_cluster"])
rank_pred             = to_1d_labels(predictions["rank"])

# 결과 데이터프레임 생성
results_df = pd.DataFrame({
    "user_id": df["user_id"].values,
    "function_id": df["function_id"].values,
    "exposure": exposure_pred,
    "ui_type": ui_type_pred,
    "service_cluster": service_cluster_pred,
    "rank": rank_pred,
})

# 사용자 타입 정보 추가 (is_senior)
user_meta = df.groupby("user_id")["is_senior"].first().reset_index()
results_df = results_df.merge(user_meta, on="user_id", how="left")

# 모듈화된 함수로 컴포넌트 타입 결정
results_df["component_type"] = results_df.apply(
    lambda row: get_component_type_by_user_type(
        row["ui_type"], 
        row["is_senior"]
    ), axis=1
)

# 서비스 클러스터 매핑
svc_map_raw = model_config["service_clusters"]["labels"]
svc_map = {str(k): v for k, v in svc_map_raw.items()}
results_df["service_cluster"] = results_df["service_cluster"].astype(str).map(svc_map)

print("추론 완료")
print(f"시니어 사용자: {results_df['is_senior'].sum()}명")
print(f"일반 사용자: {len(results_df) - results_df['is_senior'].sum()}명")

## 8. UI 그룹화 및 최종 결과 생성

In [ ]:
print("UI 그룹화 및 최종 결과 생성 중...")

# 노출될 기능만 필터링
exposure_threshold = 0.7
exposed_functions = results_df[results_df["exposure"] > exposure_threshold].copy()

# 아이콘 추천
print("아이콘 추천 생성 중...")
exposed_functions["icon_suggestion"] = exposed_functions.apply(
    lambda row: get_icon_suggestion(row["service_cluster"]), axis=1
)

# 배치 처리를 위한 데이터 준비
user_ids = exposed_functions["user_id"].unique()
user_data_batches = []

for user_id in user_ids:
    user_functions = exposed_functions[exposed_functions["user_id"] == user_id].copy()
    user_functions = user_functions.sort_values("rank")
    user_data_batches.append({
        "user_id": user_id,
        "is_senior": user_functions["is_senior"].iloc[0],
        "functions": user_functions.to_dict('records')
    })

# 배치 단위로 UI 컴포넌트 그룹 생성 
final_results = []
batch_size = 32

for i in range(0, len(user_data_batches), batch_size):
    batch = user_data_batches[i:i + batch_size]
    
    # 배치 내에서 병렬 처리
    batch_results = []
    for user_data in batch:
        try:
            # UI 컴포넌트 그룹 생성
            groups = create_ui_component_groups(user_data["functions"], user_data["user_id"])
            
            user_result = {
                "user_id": user_data["user_id"],
                "is_senior": user_data["is_senior"],
                "home": {
                    "groups": groups
                }
            }
            batch_results.append(user_result)
            
        except Exception as e:
            print(f"사용자 {user_data['user_id']} 처리 중 오류: {e}")
            continue
    
    final_results.extend(batch_results)
    
    # 진행상황 표시용
    progress = min(i + batch_size, len(user_data_batches))
    print(f"진행률: {progress}/{len(user_data_batches)} ({progress/len(user_data_batches)*100:.1f}%)")

print(f"UI 그룹화 완료: {len(final_results)}명 사용자")
print(f"시니어 사용자: {sum(1 for r in final_results if r['is_senior'])}명")
print(f"일반 사용자: {sum(1 for r in final_results if not r['is_senior'])}명")

## 9. 결과 저장 및 다운로드

In [ ]:
import json
import numpy as np

# JSON 직렬화를 위한 데이터 정리 함수
def clean_for_json(obj):
    """JSON 직렬화를 위해 numpy 타입을 Python 기본 타입으로 변환"""
    if isinstance(obj, np.integer):
        return int(obj)
    elif isinstance(obj, np.floating):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    elif isinstance(obj, np.bool_):
        return bool(obj)
    elif isinstance(obj, dict):
        return {key: clean_for_json(value) for key, value in obj.items()}
    elif isinstance(obj, list):
        return [clean_for_json(item) for item in obj]
    else:
        return obj

# final_results 데이터 정리
cleaned_results = clean_for_json(final_results)

# JSON 파일로 저장
output_file = "ui_rec/data/outputs/ui_home_outputs.json"
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(cleaned_results, f, ensure_ascii=False, indent=2)

print(f"결과 저장 완료: {output_file}")

print("Personalized UI Recommendation System 실행 완료")
print(f"총 사용자: {len(cleaned_results)}명")

In [ ]:
# 코랩에서 파일 다운로드
# files.download(output_file)